In [ ]:
import TheCannon
from TheCannon import dataset

import os

import numpy as np

import astropy.io.fits as pyfits
import astropy.io.ascii as ascii

from TheCannon.apogee import get_pixmask

from tqdm.notebook import trange, tqdm

from figsave import savefig as savefig

import matplotlib.pyplot as plt

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": 'serif',
    "font.serif": ["Computer Modern"],
    "font.size": 10,
    "figure.dpi": 200
})

In [ ]:
def wl_to_idx(wl, wl_arr):
    return int((wl - wl_arr[0]) / (wl_arr[1] - wl_arr[0]))

def get_ranges(wl):
    factor = len(wl)/313230
    return [[0, int(152262 * factor)], [int(155557 * factor), len(wl) - 1]]

In [ ]:
def load_spectrum(file): 
    with pyfits.open(file) as fits:
        target = fits[1].header["TITLE"]
        wl = fits[1].data[0][0]
        flux = fits[1].data[0][1]

        scalefactor = fits[0].header["SNR"] / np.nanmedian(np.sqrt(flux))
        snr = scalefactor * np.sqrt(flux.ravel())
        std = flux / snr

        badpix = get_pixmask(flux, std)
        ivar = np.zeros(len(flux))
        ivar[~badpix] = 1.0 / std[~badpix]**2

    return (target, wl, flux, ivar)

data_dir = '../data/fits_SNR0'

files = list(sorted([data_dir + "/" + filename for filename in os.listdir(data_dir) if not filename.startswith('.') ]))
nstars = len(files)

data = np.array([ load_spectrum(file) for file in tqdm(files) ], dtype=object)

In [ ]:
targets = np.array(data[:,0])
wl = np.array(data[:,1], dtype=object)
flux = np.array(data[:,2], dtype=object)
ivar = np.array(data[:,3], dtype=object)

In [ ]:
label_data = ascii.read('../data/HARPS_masterlist_use.csv')
label_ids = label_data['Star']
label_inds = label_ids.argsort()
label_ids = label_ids[label_inds]

label_headers = ('Teff', 'logg', 'vsini',
                 '[Fe/H]_master', '[Na/H]c_master',
                 '[Mg/H]_master', '[Al/H]c_master',
                 '[Si/H]_master', '[Ca/H]_master',
                 '[V/H]c_master', '[Mn/H]_master',
                 '[Co/H]c_master', '[O/H]_master',
                 '[Ni/H]_master', '[C/H]_master',
                 '[ScI/H]c_master', '[TiI/H]c_master',
                 '[CrI/H]_master', '[YII/H]_DM17',
                 '[S/H]_master')

labels = np.array([ label_data[header] for header in label_headers ])[:,label_inds].T

In [ ]:
#print(np.where(np.array(list(map(lambda a: 'CD-436810'.startswith(a), label_ids))) == True)[0])
#print(np.where(np.array(list(map(lambda a: 'r'.startswith(a), label_ids))) == True)[0])

twl = np.nonzero([ len(np.where(np.array(list(map(lambda a: t.startswith(a), label_ids))) == True)[0]) for t in tqdm(targets) ])
tls = [ np.where(np.array(list(map(lambda a: t.startswith(a), label_ids))) == True)[0][0] for t in tqdm(targets[twl]) ]

targets = targets[twl]
wl = wl[twl]
flux = flux[twl]
ivar = ivar[twl]

labels = labels[tls]

In [ ]:
min_wl = min([ k[0] for k in wl ])
max_wl = max([ k[len(k) - 1] for k in wl ])

new_wl = np.arange(min_wl, max_wl, 0.01)
    
flux = np.array([ np.interp(new_wl, wl[i], flux[i], left=0, right=0)
    for i in trange(0, len(wl)) ])

ivar = np.array([ np.interp(new_wl, wl[i], ivar[i], left=0, right=0)
    for i in trange(0, len(wl)) ])
    
wl = new_wl

flux_orig = np.copy(flux[0:4])

In [ ]:
template_shift = 0

flux[0] = np.interp(wl * (1 + template_shift), wl, flux[0], left=0, right=0)
ivar[0] = np.interp(wl * (1 + template_shift), wl, ivar[0], left=0, right=0)

In [ ]:
def align_spectrum(wl, flux, ivar, t_flux, lines = (np.array(range(0,18)) * 100 + 4000), width = 30):
    
    wl_ranges = np.array(
            [ wl[wl_to_idx(line - width, wl):wl_to_idx(line + width, wl)]
                for line in lines ])
    
    flux_ranges = np.array(
            [ flux[wl_to_idx(line - width, wl):wl_to_idx(line + width, wl)]
                for line in lines ])
    t_flux_ranges = np.array(
            [ t_flux[wl_to_idx(line - width, wl):wl_to_idx(line + width, wl)]
                for line in lines ])
    z = np.median([ 
        (np.correlate(flux_ranges[i] - flux_ranges[i].mean(),
        t_flux_ranges[i] - t_flux_ranges[i].mean(), mode="full").argmax() -
        len(flux_ranges[i])) * 0.01 / lines[i]
        
        for i in range(0, len(lines))
    ])

    new_flux = np.interp(wl * (1 + z), wl, flux, left=0, right=0)
    new_ivar = np.interp(wl * (1 + z), wl, ivar, left=0, right=0)

    return new_flux, new_ivar
    
for i in trange(1, len(flux)):
    flux[i], ivar[i] = align_spectrum(wl, flux[i], ivar[i], flux[0])

In [ ]:

def bin_flux(flux, ivar):
    if np.sum(ivar)==0:
        return np.mean(flux)
    return np.average(flux, weights=ivar)

def downsample_wl(wl, amount):

    discard = len(wl) % amount

    if discard != 0:
        wl = np.delete(wl, range(-discard - 1, -1))

    wl = wl.reshape(-1, amount)

    wl_b = np.mean(wl, axis=1)

    return wl_b

def downsample_spectrum(flux, ivar, amount):

    discard = len(flux) % amount

    if discard != 0:
        flux = np.delete(flux, range(-discard - 1, -1))
        ivar = np.delete(ivar, range(-discard - 1, -1))

    ivar = ivar.reshape(-1, amount)
    flux = flux.reshape(-1, amount)

    ivar_b = np.sqrt(np.sum(ivar**2, axis=1))
    flux_b = np.array([bin_flux(f, w) for f,w in zip(flux, ivar)])

    return (flux_b, ivar_b)


dwl = downsample_wl(wl, 10)

downsampled = np.array([ downsample_spectrum(flux[i], ivar[i], 10) for i in trange(0, len(flux)) ])

dflux = downsampled[:,0]
divar = downsampled[:,1]

In [ ]:
ds = dataset.Dataset(dwl, targets, dflux, divar, labels, np.array(['']), np.array([[0] * len(dwl)]), np.array([[0] * len(dwl)]))

ds.ranges = get_ranges(dwl)

p_flux, p_ivar = ds.continuum_normalize_training_q(q=0.9, delta_lambda=50)

contmask = ds.make_contmask(p_flux, p_ivar, frac=0.07)

ds.set_continuum(contmask)

cont = ds.fit_continuum(3, "sinusoid")

norm_flux, norm_ivar, _, _ = ds.continuum_normalize(cont)

dcont_flux = dflux/norm_flux
dcont_ivar = divar/norm_ivar

cont_flux = np.array([ np.interp(wl, dwl, dcont_flux[i]) for i in range(0, len(dcont_flux)) ])
cont_ivar = np.array([ np.interp(wl, dwl, dcont_ivar[i]) for i in range(0, len(dcont_ivar)) ])

flux = flux/cont_flux
ivar = ivar/cont_ivar

ivar[np.isnan(flux)] = 0
ivar[np.isnan(ivar)] = 0
flux[np.isnan(flux)] = 0


In [ ]:
import pickle as pkl

with open("data_small.pkl", "wb") as f:
    pkl.dump((wl, targets, flux, ivar, labels), f)

In [ ]:
wl = downsample_wl(wl, 2)

In [ ]:
flux, ivar = np.array([ downsample_spectrum(flux[i], ivar[i], 2) for i in trange(0, len(flux)) ])